In [1]:
import sys
sys.path.append('../deepsurv')
import deepsurv

#from deepsurv_logger import DeepSurvLogger, TensorboardLogger
import utils
import viz
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import lasagne
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [2]:
eln_clin_demo_comp = [0]+list(range(153,180))
eln_cyto_gen_comp = list(range(171))
eln_cyto_comp = [0] + list(range(84,171))
eln_gen_comp = list(range(84)) + list(range(153,171))
clin_demo_comp = list(range(153,180))
cyto_gen_comp = list(range(1,171))
cyto_comp = list(range(84,171))
gen_comp = list(range(1,84))+list(range(153,171))
clin_demo_cyto_gen_comp = list(range(1,180))
comp = list(range(153,171))

In [3]:
train_df = pd.read_table('df_prognosis_features_ready_final_component.tsv').iloc[:,clin_demo_cyto_gen_comp+[180,181]]
train,test = train_test_split(train_df,test_size=0.2,random_state=17)
train,val = train_test_split(train,test_size=0.2,random_state=17)

In [5]:
def dataframe_to_deepsurv_ds(df, event_col = 'os_status', time_col = 'os'):
    # Extract the event and time columns as numpy arrays
    e = df[event_col].values.astype(np.int32)
    t = df[time_col].values.astype(np.float32)

    # Extract the patient's covariates as a numpy array
    x_df = df.drop([event_col, time_col], axis = 1)
    x = x_df.values.astype(np.float32)
    #x=x_df
    
    # Return the deep surv dataframe
    return {
        'x' : x,
        'e' : e,
        't' : t
    }

In [6]:
### Min Max Scaler Normalization and good format of the data to feed the MLP network
scale = preprocessing.MinMaxScaler().fit(train)
df_train = pd.DataFrame(scale.transform(train.values), columns=train.columns, index=train.index)
df_val = pd.DataFrame(scale.transform(val.values), columns=val.columns, index=val.index)
df_test = pd.DataFrame(scale.transform(test.values), columns=test.columns, index=test.index)
train_data = dataframe_to_deepsurv_ds(df_train, event_col = 'os_status', time_col= 'os')
val_data = dataframe_to_deepsurv_ds(df_val, event_col = 'os_status', time_col= 'os')
test_data = dataframe_to_deepsurv_ds(df_test, event_col = 'os_status', time_col= 'os')

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [15]:
hid_layers=[[100,100,10],[500,500,10],[250,100,100],[200,100,10]]
drop=[0.2,0.2,0.2,0.2]
network=[None]*4
metrics=[None]*4
for i in range(4):

    hyperparams = {
        'L2_reg': 10.0,
        'batch_norm': True,
        'dropout': 0.2,
        'hidden_layers_sizes': hid_layers[i],
        'learning_rate': 1e-05,
        'lr_decay': 0.001,
        'momentum': 0.9,
        'n_in': train_data['x'].shape[1],
        'activation':'rectify'
        #'standardize': True
    }
    np.random.seed(17)
    network[i] = deepsurv.DeepSurv(**hyperparams)
    network[i].restored_update_params = None 
    logger = deepsurv.deepsurv_logger.TensorboardLogger(name = 'DeepSurvLogger', logdir = './logs/tensorboard/')
    np.random.seed(17)
    #update_fn=lasagne.updates.nesterov_momentum # The type of optimizer to use. 
    update_fn=lasagne.updates.rmsprop
    n_epochs = 12000
    val_frequency=250
    patience=10000 # minimum number of epochs to train for
    metrics[i] =  network[i].train(train_data,val_data, n_epochs=n_epochs, logger=logger, update_fn=update_fn,patience=patience,validation_frequency=val_frequency)

2019-05-29 12:34:23,057 - Training step 0/12000     |                         | - loss: 33.9533 - ci: 0.4849
2019-05-29 12:34:23,057 - Training step 0/12000     |                         | - loss: 33.9533 - ci: 0.4849
2019-05-29 12:34:23,057 - Training step 0/12000     |                         | - loss: 33.9533 - ci: 0.4849
2019-05-29 12:34:23,057 - Training step 0/12000     |                         | - loss: 33.9533 - ci: 0.4849
2019-05-29 12:34:23,057 - Training step 0/12000     |                         | - loss: 33.9533 - ci: 0.4849
2019-05-29 12:34:23,057 - Training step 0/12000     |                         | - loss: 33.9533 - ci: 0.4849
2019-05-29 12:34:23,057 - Training step 0/12000     |                         | - loss: 33.9533 - ci: 0.4849
2019-05-29 12:34:23,057 - Training step 0/12000     |                         | - loss: 33.9533 - ci: 0.4849
2019-05-29 12:34:23,057 - Training step 0/12000     |                         | - loss: 33.9533 - ci: 0.4849
2019-05-29 12:34:23

In [11]:
for i in range(4):
    print (network[i].get_concordance_index(x=val_data['x'],t=val_data['t'],e=val_data['e']),network[i].get_concordance_index(x=test_data['x'],t=test_data['t'],e=test_data['e']),network[i].hyperparams)

0.672710042644455 0.7062294292365738 {'n_in': 179, 'learning_rate': 1e-05, 'hidden_layers_sizes': [100, 100, 10], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10.0, 'L1_reg': 0.0, 'activation': 'rectify', 'dropout': 0.4, 'batch_norm': True, 'standardize': False}
0.658520162597571 0.7132375371277193 {'n_in': 179, 'learning_rate': 1e-05, 'hidden_layers_sizes': [500, 500, 10], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10.0, 'L1_reg': 0.0, 'activation': 'rectify', 'dropout': 0.4, 'batch_norm': True, 'standardize': False}
0.6693683134242749 0.7229991169623505 {'n_in': 179, 'learning_rate': 1e-05, 'hidden_layers_sizes': [250, 100, 100], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10.0, 'L1_reg': 0.0, 'activation': 'rectify', 'dropout': 0.4, 'batch_norm': True, 'standardize': False}
0.6662510287039577 0.713205426667737 {'n_in': 179, 'learning_rate': 1e-05, 'hidden_layers_sizes': [200, 100, 10], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10.0, 'L1_reg': 0.0, 'activation': 'rectify', 

In [16]:
for i in range(4):
    print (network[i].get_concordance_index(x=val_data['x'],t=val_data['t'],e=val_data['e']),network[i].get_concordance_index(x=test_data['x'],t=test_data['t'],e=test_data['e']),network[i].hyperparams)

0.6621985585675453 0.7005779882796821 {'n_in': 179, 'learning_rate': 1e-05, 'hidden_layers_sizes': [100, 100, 10], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10.0, 'L1_reg': 0.0, 'activation': 'rectify', 'dropout': 0.2, 'batch_norm': True, 'standardize': False}
0.6316117608917928 0.7000642209199647 {'n_in': 179, 'learning_rate': 1e-05, 'hidden_layers_sizes': [500, 500, 10], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10.0, 'L1_reg': 0.0, 'activation': 'rectify', 'dropout': 0.2, 'batch_norm': True, 'standardize': False}
0.6499663333250206 0.7079072007706511 {'n_in': 179, 'learning_rate': 1e-05, 'hidden_layers_sizes': [250, 100, 100], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10.0, 'L1_reg': 0.0, 'activation': 'rectify', 'dropout': 0.2, 'batch_norm': True, 'standardize': False}
0.6509888027132846 0.7103154852693265 {'n_in': 179, 'learning_rate': 1e-05, 'hidden_layers_sizes': [200, 100, 10], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 10.0, 'L1_reg': 0.0, 'activation': 'rectify